In [2]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.types import Send

class ConversationState(TypedDict):
    """실시간 상담 에이전트의 상태"""
    
    # === WebRTC 스트림 ===
    session_id: str                     # WebSocket 세션 ID
    is_streaming: bool                  # 스트리밍 활성 상태
    audio_chunks: list[bytes]           # 누적 오디오 청크
    chunk_timestamps: list[float]       # 각 청크의 타임스탬프
    
    # === 전사 ===
    transcription: str                  # 현재 전사 텍스트
    full_transcript: list[str]          # 누적 전사 히스토리
    
    # === 요약 (병렬 처리) ===
    current_summary: str                # 현재까지의 대화 요약
    
    # === RAG (병렬 처리) ===
    query: str                          # RAG 검색 쿼리
    retrieved_docs: list[str]           # 검색된 문서
    suggestion: str                     # 생성된 답변 추천
    
    # === 메타데이터 ===
    timestamp: float
    speaker: str                        # 'agent' | 'customer'

In [3]:
def transcribe_node(state: ConversationState) -> ConversationState:
    """
    음성 → 텍스트 변환 (STT)
    실제 구현: STT API 호출 (Google Cloud STT, Deepgram 등)
    """
    print("🎤 [transcribe_node] 실행 중...")
    
    # TODO: 실제 STT API 호출
    # transcription = stt_api.transcribe(state['audio_chunks'][-1])
    
    # Placeholder 구현
    transcription = f"[전사 텍스트 #{len(state.get('full_transcript', []))}]"
    
    return {
        **state,
        "transcription": transcription,
        "full_transcript": state.get("full_transcript", []) + [transcription],
        "timestamp": state.get("timestamp", 0.0) + 0.5
    }


def summarize_node(state: ConversationState) -> ConversationState:
    """
    대화 요약 생성 (병렬 처리)
    실제 구현: LLM 요약 모델 호출
    """
    print("📝 [summarize_node] 실행 중... (병렬)")
    
    # TODO: 실제 요약 LLM 호출
    # summary = llm.summarize(state['full_transcript'])
    
    # Placeholder 구현
    transcript_count = len(state.get("full_transcript", []))
    summary = f"[대화 요약: {transcript_count}개 발화 분석 완료]"
    
    return {
        **state,
        "current_summary": summary
    }


def rag_retrieve_node(state: ConversationState) -> ConversationState:
    """
    RAG 기반 지식 검색 (병렬 처리)
    실제 구현: Vector DB 검색 (Pinecone, Chroma 등)
    """
    print("🔍 [rag_retrieve_node] 실행 중... (병렬)")
    
    # TODO: 실제 Vector DB 검색
    # docs = vector_db.search(state['transcription'], top_k=3)
    
    # Placeholder 구현
    query = state.get("transcription", "")
    docs = [
        f"[문서 1: {query}에 대한 관련 내용]",
        f"[문서 2: {query}에 대한 참고 자료]",
        f"[문서 3: {query}에 대한 FAQ]"
    ]
    
    return {
        **state,
        "query": query,
        "retrieved_docs": docs
    }


def generate_suggestion_node(state: ConversationState) -> ConversationState:
    """
    RAG 기반 답변 추천 생성
    실제 구현: LLM 생성 모델 호출
    """
    print("💡 [generate_suggestion_node] 실행 중...")
    
    # TODO: 실제 LLM 생성
    # suggestion = llm.generate(
    #     context=state['retrieved_docs'],
    #     query=state['transcription']
    # )
    
    # Placeholder 구현
    docs_summary = f"{len(state.get('retrieved_docs', []))}개 문서"
    suggestion = f"[추천 답변: {docs_summary} 기반 생성]"
    
    return {
        **state,
        "suggestion": suggestion
    }


def aggregate_node(state: ConversationState) -> ConversationState:
    """
    병렬 처리 결과 통합 (Barrier)
    """
    print("🔄 [aggregate_node] 병렬 결과 통합 중...")
    
    # 병렬 처리 결과 확인
    summary = state.get("current_summary", "없음")
    suggestion = state.get("suggestion", "없음")
    
    print(f"  ├─ 요약: {summary}")
    print(f"  └─ 추천: {suggestion}")
    
    return state

In [ ]:
def create_graph():
    """실시간 상담 에이전트 그래프 생성"""
    
    graph = StateGraph(ConversationState)
  
    graph.add_node("transcribe", transcribe_node)
    graph.add_node("summarize", summarize_node)
    graph.add_node("rag_retrieve", rag_retrieve_node)
    graph.add_node("generate_suggestion", generate_suggestion_node)
    graph.add_node("aggregate", aggregate_node)
    
    # START → transcribe
    graph.add_edge(START, "transcribe")
    
    # transcribe → 병렬 분기 [summarize, rag_retrieve]
    graph.add_edge("transcribe", "summarize")
    graph.add_edge("transcribe", "rag_retrieve")
    
    # 병렬 경로 → aggregate (Barrier)
    graph.add_edge("summarize", "aggregate")
    
    # rag_retrieve → generate_suggestion → aggregate
    graph.add_edge("rag_retrieve", "generate_suggestion")
    graph.add_edge("generate_suggestion", "aggregate")
    
    # aggregate → END
    graph.add_edge("aggregate", END)
    
    return graph

# 그래프 생성
workflow = create_graph()
graph = workflow.compile()

print("✅ 그래프 구조 정의 완료")
print("\n📊 그래프 구조:")
print("""
START
  ↓
transcribe_node (음성 → 텍스트)
  ↓
  ├──────────────────────┐
  ↓                      ↓
summarize_node      rag_retrieve_node (병렬)
  ↓                      ↓
  ↓              generate_suggestion_node
  ↓                      ↓
  └──────→ aggregate_node ←──
               ↓
              END
""")

✅ 그래프 구조 정의 완료

📊 그래프 구조:

START
  ↓
transcribe_node (음성 → 텍스트)
  ↓
  ├──────────────────────┐
  ↓                      ↓
summarize_node      rag_retrieve_node (병렬)
  ↓                      ↓
  ↓              generate_suggestion_node
  ↓                      ↓
  └──────→ aggregate_node ←──
               ↓
              END



In [ ]:
import time

# 초기 상태 생성
initial_state: ConversationState = {
    "session_id": "test-session-001",
    "is_streaming": True,
    "audio_chunks": [b"fake_audio_data_1"],
    "chunk_timestamps": [time.time()],
    "transcription": "",
    "full_transcript": [],
    "current_summary": "",
    "query": "",
    "retrieved_docs": [],
    "suggestion": "",
    "timestamp": time.time(),
    "speaker": "customer"
}

print("=" * 60)
print("🚀 그래프 실행 시작")
print("=" * 60)

# 그래프 실행
result = graph.invoke(initial_state)

print("\n" + "=" * 60)
print("✅ 그래프 실행 완료")
print("=" * 60)

# 결과 출력
print("\n📋 최종 결과:")
print(f"  전사: {result['transcription']}")
print(f"  요약: {result['current_summary']}")
print(f"  추천: {result['suggestion']}")
print(f"  검색된 문서: {len(result['retrieved_docs'])}개")

In [ ]:
# ============================================
# 6. 그래프 시각화 (선택적)
# ============================================

try:
    from IPython.display import Image, display
    
    # Mermaid 다이어그램 생성
    graph_image = app.get_graph().draw_mermaid_png()
    
    # 이미지 표시
    display(Image(graph_image))
    print("✅ 그래프 시각화 완료")
    
except Exception as e:
    print(f"⚠️ 시각화 실패: {e}")
    print("   대안: app.get_graph().print_ascii() 사용")
    print("\n" + app.get_graph().print_ascii())

In [ ]:
# ============================================
# 7. 다중 청크 처리 시뮬레이션
# ============================================

print("🔁 다중 청크 처리 시뮬레이션 (3회)")
print("=" * 60)

# 초기 상태
state = {
    "session_id": "test-session-002",
    "is_streaming": True,
    "audio_chunks": [],
    "chunk_timestamps": [],
    "transcription": "",
    "full_transcript": [],
    "current_summary": "",
    "query": "",
    "retrieved_docs": [],
    "suggestion": "",
    "timestamp": time.time(),
    "speaker": "customer"
}

# 3개 청크 순차 처리
for i in range(3):
    print(f"\n{'='*60}")
    print(f"📦 청크 #{i+1} 처리 중...")
    print(f"{'='*60}")
    
    # 새 오디오 청크 추가
    state["audio_chunks"].append(f"audio_chunk_{i+1}".encode())
    state["chunk_timestamps"].append(time.time())
    
    # 그래프 실행
    state = app.invoke(state)
    
    # 진행 상황 출력
    print(f"\n  전사 히스토리: {len(state['full_transcript'])}개")
    print(f"  최신 전사: {state['transcription']}")
    print(f"  현재 요약: {state['current_summary']}")
    print(f"  최신 추천: {state['suggestion']}")

print(f"\n{'='*60}")
print("✅ 다중 청크 처리 완료")
print(f"{'='*60}")
print(f"\n📊 최종 통계:")
print(f"  처리한 청크: {len(state['audio_chunks'])}개")
print(f"  전사 히스토리: {len(state['full_transcript'])}개")
print(f"  최종 요약: {state['current_summary']}")
print(f"  최종 추천: {state['suggestion']}")

# 📝 다음 구현 단계

## 현재 상태
✅ LangGraph 구조 완성 (병렬 처리 포함)  
✅ State 정의 완료  
✅ 5개 노드 Placeholder 구현  
⏳ WebSocket 서버 (미구현)  
⏳ 실제 STT/LLM/RAG 통합 (미구현)

---

## 구현해야 할 기능

### 1. 노드 실제 구현
- **transcribe_node**: STT API 통합 (Google Cloud STT, Deepgram, Whisper)
- **summarize_node**: LLM 요약 (OpenAI, Anthropic, Ollama)
- **rag_retrieve_node**: Vector DB 검색 (Pinecone, Chroma, FAISS)
- **generate_suggestion_node**: LLM 생성 (RAG 기반)

### 2. WebSocket 서버
```python
# FastAPI WebSocket Handler
@app.websocket("/stt")
async def websocket_endpoint(websocket: WebSocket):
    # 오디오 청크 수신
    # LangGraph Agent 실행
    # 실시간 결과 푸시
```

### 3. 클라이언트 (JavaScript)
```javascript
// AudioWorklet으로 오디오 추출
// WebSocket으로 서버 전송
// 실시간 결과 수신 및 UI 업데이트
```

---

## 병렬 처리 확인
현재 그래프는 **transcribe → [summarize || rag_retrieve]** 구조로 병렬 처리를 지원합니다.

실행 흐름:
1. transcribe_node 실행
2. **병렬 분기**: summarize_node & rag_retrieve_node 동시 실행
3. generate_suggestion_node 실행 (rag_retrieve 결과 사용)
4. **Barrier**: aggregate_node가 모든 결과 대기
5. 통합 결과 반환

---

## 테스트 방법
```python
# 단일 청크 테스트
result = app.invoke(initial_state)

# 다중 청크 테스트 (위의 셀 7 실행)
# 각 청크마다 그래프가 실행되며 상태가 누적됩니다
```